In [2]:
import pandas as pd
import ast # Buat nanganin data yang bentuknya kayak list dalam string

df = pd.read_csv('movies.csv')
# Kita pilih kolom yang paling ngaruh ke "vibe" film
df = df[['index', 'title', 'overview', 'genres', 'keywords', 'cast', 'crew']]
df.head()

,index,title,overview,genres,keywords,cast,crew
0,0,Avatar,"In the 22nd century, a paraplegic Marine is di...",Action Adventure Fantasy Science Fiction,culture clash future space war space colony so...,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd..."
1,1,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",Adventure Fantasy Action,ocean drug abuse exotic island east india trad...,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa..."
2,2,Spectre,A cryptic message from Bond’s past sends him o...,Action Adventure Crime,spy based on novel secret agent sequel mi6,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar..."
3,3,The Dark Knight Rises,Following the death of District Attorney Harve...,Action Crime Drama Thriller,dc comics crime fighter terrorist secret ident...,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm..."
4,4,John Carter,"John Carter is a war-weary, former military ca...",Action Adventure Science Fiction,based on novel mars medallion space travel pri...,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa..."


In [4]:
import ast

def convert_to_list(obj):
    # Jika data kosong (NaN)
    if pd.isna(obj):
        return []
    # Jika data diawali '[' berarti format JSON/List-string
    if isinstance(obj, str) and obj.startswith('['):
        try:
            L = []
            for i in ast.literal_eval(obj):
                L.append(i['name'])
            return L
        except:
            return []
    # Jika data sudah string biasa (seperti error kamu: "Action Adventure Fantasy")
    elif isinstance(obj, str):
        return obj.split()
    return []

def fetch_director(obj):
    if pd.isna(obj) or not obj.startswith('['):
        return []
    try:
        L = []
        for i in ast.literal_eval(obj):
            if i['job'] == 'Director':
                L.append(i['name'])
                break
        return L
    except:
        return []

# Eksekusi pembersihan dengan logika baru
df['genres'] = df['genres'].apply(convert_to_list)
df['keywords'] = df['keywords'].apply(convert_to_list)
df['cast'] = df['cast'].apply(lambda x: convert_to_list(x)[:3]) # Ambil 3 aktor saja
df['director'] = df['crew'].apply(fetch_director)

# Beresin Overview (ubah string jadi list kata)
df['overview'] = df['overview'].apply(lambda x: x.split() if isinstance(x, str) else [])

df.head()

,index,title,overview,genres,keywords,cast,crew,director
0,0,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin...","[Action, Adventure, Fantasy, Science, Fiction]","[culture, clash, future, space, war, space, co...","[Sam, Worthington, Zoe]","[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",[James Cameron]
1,1,Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, d...","[Adventure, Fantasy, Action]","[ocean, drug, abuse, exotic, island, east, ind...","[Johnny, Depp, Orlando]","[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",[Gore Verbinski]
2,2,Spectre,"[A, cryptic, message, from, Bond’s, past, send...","[Action, Adventure, Crime]","[spy, based, on, novel, secret, agent, sequel,...","[Daniel, Craig, Christoph]","[{'name': 'Thomas Newman', 'gender': 2, 'depar...",[Sam Mendes]
3,3,The Dark Knight Rises,"[Following, the, death, of, District, Attorney...","[Action, Crime, Drama, Thriller]","[dc, comics, crime, fighter, terrorist, secret...","[Christian, Bale, Michael]","[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",[Christopher Nolan]
4,4,John Carter,"[John, Carter, is, a, war-weary,, former, mili...","[Action, Adventure, Science, Fiction]","[based, on, novel, mars, medallion, space, tra...","[Taylor, Kitsch, Lynn]","[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",[Andrew Stanton]


In [5]:
df['genres'] = df['genres'].apply(lambda x: [i.replace(" ","") for i in x])
df['keywords'] = df['keywords'].apply(lambda x: [i.replace(" ","") for i in x])
df['cast'] = df['cast'].apply(lambda x: [i.replace(" ","") for i in x])
df['director'] = df['director'].apply(lambda x: [i.replace(" ","") for i in x])


In [8]:
# Gabungkan semua list menjadi satu kolom tags
df['tags'] = df['overview'] + df['genres'] + df['keywords'] + df['cast'] + df['director']

# Buat DataFrame baru dengan .copy() untuk menghindari SettingWithCopyWarning
new_df = df[['index', 'title', 'tags']].copy()

# Ubah list menjadi string (paragraf) dan kecilkan hurufnya
new_df['tags'] = new_df['tags'].apply(lambda x: " ".join(x).lower())

# Cek hasil akhir
print(new_df['tags'][0])
new_df.head()

in the 22nd century, a paraplegic marine is dispatched to the moon pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization. action adventure fantasy science fiction culture clash future space war space colony society sam worthington zoe jamescameron


,index,title,tags
0,0,Avatar,"in the 22nd century, a paraplegic marine is di..."
1,1,Pirates of the Caribbean: At World's End,"captain barbossa, long believed to be dead, ha..."
2,2,Spectre,a cryptic message from bond’s past sends him o...
3,3,The Dark Knight Rises,following the death of district attorney harve...
4,4,John Carter,"john carter is a war-weary, former military ca..."


In [9]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Ambil 5000 kata paling penting, buang kata hubung (stop words)
cv = CountVectorizer(max_features=5000, stop_words='english')
vectors = cv.fit_transform(new_df['tags']).toarray()

# Hitung jarak antar film (Semakin dekat angkanya ke 1, semakin mirip)
similarity = cosine_similarity(vectors)

In [10]:
def recommend(movie):
    movie_index = new_df[new_df['title'] == movie].index[0]
    distances = similarity[movie_index]
    # Urutin dari yang paling mirip
    movies_list = sorted(list(enumerate(distances)), reverse=True, key=lambda x: x[1])[1:6]
    
    for i in movies_list:
        print(new_df.iloc[i[0]].title)

recommend('Batman Begins') # Ganti judulnya buat ngetes

The Dark Knight
The Dark Knight Rises
Batman
Batman & Robin
Amidst the Devil's Wings


In [12]:
def get_movies_by_director(director_name):
    # Hilangkan spasi agar cocok dengan format di dataset (misal: James Cameron -> JamesCameron)
    director_name = director_name.replace(" ", "")
    
    # Filter dataframe berdasarkan kolom director
    # Kita pakai dataset 'df' yang masih lengkap (bukan new_df) 
    # karena new_df hanya berisi id, title, dan tags
    director_movies = df[df['director'].apply(lambda x: director_name in x)]
    
    if director_movies.empty:
        print(f"Maaf, sutradara '{director_name}' tidak ditemukan.")
    else:
        print(f"Film karya {director_name}:")
        for title in director_movies['title']:
            print("-", title)

# Contoh penggunaan:
get_movies_by_director("Stanley Kubrick")

Film karya StanleyKubrick:
- Eyes Wide Shut
- Lolita
- The Shining
- Barry Lyndon
- 2001: A Space Odyssey
- Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb


In [14]:
import pandas as pd

df = pd.read_csv('movies.csv')

# Melihat 5 baris pertama
df.head()

# Melihat 5 baris secara acak (bagus buat cek variasi data)
df.sample(10)

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
4495,4495,609000,Comedy Music Romance,NaN,3080,venice dance musical tap dancing stage show,en,Top Hat,Showman Jerry Travers is working for producer ...,3.898498,...,101.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,They're Dancing Cheek-To-Cheek Again!,Top Hat,7.4,59,Fred Astaire Ginger Rogers Edward Everett Hort...,"[{'name': 'Robert Wise', 'gender': 2, 'departm...",Mark Sandrich
1192,1192,40000000,Action Adventure Fantasy Horror Science Fiction,NaN,10336,anti hero secret agent fistfight based on comi...,en,Spawn,After being murdered by corrupt colleagues in ...,19.833664,...,96.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,Born in darkness. Sworn to justice.,Spawn,5.0,336,Michael Jai White Martin Sheen John Leguizamo ...,"[{'name': 'Mary Jo Slater', 'gender': 1, 'depa...",Mark A.Z. Dipp\u00e9
3625,3625,0,Comedy,NaN,21014,bar mitvah party independent film,en,Keeping Up with the Steins,All hilarity breaks loose in this heartwarming...,3.562563,...,90.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,A 13-year-old boy uses his upcoming bar mitzva...,Keeping Up with the Steins,5.2,18,Daryl Sabara Jami Gertz Jeremy Piven Cheryl Hi...,"[{'name': 'Scott Marshall', 'gender': 0, 'depa...",Scott Marshall
4155,4155,0,Action Adventure History Western,NaN,98549,based on real events,en,The Legend of Hell's Gate: An American Conspiracy,"In 1870s Texas, a ruthless bounty hunter and a...",1.002585,...,108.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Justice was delieverd by the fastest gun.,The Legend of Hell's Gate: An American Conspiracy,3.9,8,Eric Balfour Lou Taylor Pucci Henry Thomas Jam...,"[{'name': 'Sig De Miguel', 'gender': 2, 'depar...",Tanner Beard
2509,2509,15500000,Comedy Horror Science Fiction,NaN,9035,small town mutant meteor meat alien,en,Slither,"A small town is taken over by an alien plague,...",17.327279,...,95.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Horror Has A New Face,Slither,6.3,317,Nathan Fillion Don Thompson Elizabeth Banks Gr...,"[{'name': 'James Gunn', 'gender': 2, 'departme...",James Gunn
4113,4113,2000000,Comedy,http://www.thevirginityhit.com/,42889,loss of virginity teenage boy teen comedy teen...,en,The Virginity Hit,"Four guys, one camera, and their experience ch...",3.125382,...,120.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,NaN,The Virginity Hit,4.6,19,Matt Bennett Zack Pearlman Jacob Davich Justin...,"[{'name': 'Huck Botko', 'gender': 2, 'departme...",Huck Botko
1497,1497,0,Fantasy Comedy Drama,NaN,9647,holiday tv ratings comedy scrooge christmas carol,en,Scrooged,"In this modern take on Charles Dickens' ""A Chr...",21.620317,...,101.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The spirits will move you in odd and hysterica...,Scrooged,6.7,390,Bill Murray Karen Allen John Forsythe Bobcat G...,"[{'name': 'Danny Elfman', 'gender': 2, 'depart...",Richard Donner
2295,2295,0,Action Crime,NaN,244114,robbery street war gun battle rogue cop,zh,風暴,A crew of seasoned criminals led by the notori...,3.376967,...,110.0,"[{""iso_639_1"": ""zh"", ""name"": ""\u666e\u901a\u8b...",Released,NaN,Firestorm,5.9,43,Andy Lau Gordon Lam Ka-Tung Yao Chen Hu Jun Ra...,"[{'name': 'William Kong', 'gender': 2, 'depart...",Alan Yuen
443,443,80000000,Fantasy Comedy Family Romance,NaN,10202,wishes come true escapade disorder imaginary m...,en,Bedtime Stories,Skeeter Bronson is a down-on-his-luck guy who'...,23.531721,...,99.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,Whatever they dream up... he has to survive.,Bedtime Stories,5.9,901,Adam Sandler Keri Russell Guy Pearce Courteney...,"[{'name': 'Adam Shankman', 'gender': 2, 'depar...",Adam Shankman
4300,4300,1200000,Crim

In [15]:
# Melihat rata-rata, nilai min, max, dan kuartil untuk kolom angka
df.describe()


,index,budget,id,popularity,revenue,runtime,vote_average,vote_count
count,4803.000000,4.803000e+03,4803.000000,4803.000000,4.803000e+03,4801.000000,4803.000000,4803.000000
mean,2401.000000,2.904504e+07,57165.484281,21.492301,8.226064e+07,106.875859,6.092172,690.217989
std,1386.651002,4.072239e+07,88694.614033,31.816650,1.628571e+08,22.611935,1.194612,1234.585891
min,0.000000,0.000000e+00,5.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000
25%,1200.500000,7.900000e+05,9014.500000,4.668070,0.000000e+00,94.000000,5.600000,54.000000
50%,2401.000000,1.500000e+07,14629.000000,12.921594,1.917000e+07,103.000000,6.200000,235.000000
75%,3601.500000,4.000000e+07,58610.500000,28.313505,9.291719e+07,118.000000,6.800000,737.000000
max,4802.000000,3.800000e+08,459488.000000,875.581305,2.787965e+09,338.000000,10.000000,13752.000000


In [16]:
# Filter baris yang kolom 'genres'-nya mengandung kata 'Comedy'
comedy_movies = df[df['genres'].str.contains('Comedy', na=False)]

# Menampilkan hasil filter
comedy_movies.head()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
18,18,225000000,Action Comedy Science Fiction,http://www.sonypictures.com/movies/meninblack3/,41154,time travel time machine alien fictional gover...,en,Men in Black 3,Agents J (Will Smith) and K (Tommy Lee Jones) ...,52.035179,...,106.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,They are back... in time.,Men in Black 3,6.2,4160,Will Smith Tommy Lee Jones Josh Brolin Michael...,"[{'name': 'Steven Spielberg', 'gender': 2, 'de...",Barry Sonnenfeld
40,40,200000000,Animation Family Adventure Comedy,http://www.disney.go.com/cars/,49013,car race sequel comedy anthropomorphism best f...,en,Cars 2,Star race car Lightning McQueen and his pal Ma...,49.986590,...,106.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Ka-ciao!,Cars 2,5.8,2033,Owen Wilson Larry the Cable Guy Michael Caine ...,"[{'name': 'John Lasseter', 'gender': 2, 'depar...",John Lasseter
42,42,200000000,Animation Family Comedy,http://disney.go.com/toystory/,10193,hostage college toy barbie animation,en,Toy Story 3,"Woody, Buzz, and the rest of Andy's toys haven...",59.995418,...,103.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,No toy gets left behind.,Toy Story 3,7.6,4597,Tom Hanks Tim Allen Ned Beatty Joan Cusack Mic...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Lee Unkrich
55,55,185000000,Animation Adventure Comedy Family Action,http://disney.go.com/brave/#/home,62177,scotland rebel bravery kingdom archer,en,Brave,Brave is set in the mystical Scottish Highland...,125.114374,...,93.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,Change your fate.,Brave,6.7,4641,Kelly Macdonald Julie Walters Billy Connolly E...,"[{'name': 'Bill Abbott', 'gender': 2, 'departm...",Brenda Chapman
58,58,140000000,Action Comedy Crime Thriller,NaN,5174,ambassador,en,Rush Hour 3,After an attempted assassination on Ambassador...,22.571780,...,91.0,"[{""iso_639_1"": ""la"", ""name"": ""Latin""}, {""iso_6...",Released,The Rush Is On!,Rush Hour 3,6.1,783,Chris Tucker Jackie Chan Hiroyuki Sanada Max v...,"[{'name': 'Billy Weber', 'gender': 2, 'departm...",Brett Ratner


In [21]:
# 1. Filter baris yang mengandung 'Comedy'
# 2. Pilih kolom 'genres', 'title', dan 'director'
df_comedy = df[df['genres'].str.contains('Comedy', na=False)][['genres', 'title', 'director']]

# Tampilkan hasilnya
df_comedy.head(50)

,genres,title,director
18,Action Comedy Science Fiction,Men in Black 3,Barry Sonnenfeld
40,Animation Family Adventure Comedy,Cars 2,John Lasseter
42,Animation Family Comedy,Toy Story 3,Lee Unkrich
55,Animation Adventure Comedy Family Action,Brave,Brenda Chapman
58,Action Comedy Crime Thriller,Rush Hour 3,Brett Ratner
66,Animation Comedy Family Adventure,Up,Pete Docter
70,Action Adventure Comedy Science Fiction Western,Wild Wild West,Barry Sonnenfeld
73,Fantasy Comedy Family,Evan Almighty,Tom Shadyac
77,Drama Comedy Animation Family,Inside Out,Pete Docter
86,Comedy Adventure Fantasy Animation Family,Shrek Forever After,Mike Mitchell
